In [1]:
import os

In [ ]:
os.chdir('../')
%pwd

'c:\\Users\\admin\\OneDrive\\Dokumen\\Portofolio\\Data Sceince (Python)\\End-to-end'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from end_to_end_project.constants import *
from end_to_end_project.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = Path(config.root_dir),
            data_path = Path(config.data_path)
        )

        return data_transformation_config

In [8]:
import os
from end_to_end_project import logger
from sklearn.model_selection import train_test_split
import pandas as pd

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_split(self, df: pd.DataFrame, test_size: float = 0.25, random_state:int = 42):
        data = df.copy()

        train, test = train_test_split(data, test_size=test_size, random_state=random_state)
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index = False )
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index = False )

        logger.info(f"Train and test data saved at {self.config.root_dir}")
        logger.info(f"Train data shape: {train.shape}, Test data shape: {test.shape}")
        

[2025-07-31 21:25:25,285: INFO: utils: NumExpr defaulting to 4 threads.]


In [9]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_split(
        df = pd.read_csv(data_transformation_config.data_path),
        test_size= 0.25,
        random_state= 42
    )
except Exception as e:
    raise e

[2025-07-31 21:32:36,215: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-07-31 21:32:36,218: INFO: common: YAML file: params.yaml loaded successfully]
[2025-07-31 21:32:36,226: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-07-31 21:32:36,230: INFO: common: Created directory at: artifacts]
[2025-07-31 21:32:36,232: INFO: common: Created directory at: artifacts/data_transformation]
[2025-07-31 21:32:36,325: INFO: 838343783: Train and test data saved at artifacts\data_transformation]
[2025-07-31 21:32:36,326: INFO: 838343783: Train data shape: (1199, 12), Test data shape: (400, 12)]
